In [1]:
import os

os.chdir("../")

In [2]:
#!pip install /kaggle/input/polars-for-student/polars-0.16.9-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
#!pip install /kaggle/input/polars-for-student/typing_extensions-4.5.0-py3-none-any.whl

In [3]:
import gc
import os

# from input.utilcode.utils.constants import *
import pandas as pd
import numpy as np
import warnings
import pickle
import polars as pl

from collections import defaultdict
from itertools import combinations
import pyarrow as pa

from xgboost import XGBClassifier
from tsfresh.feature_extraction import feature_calculators

from lightgbm import LGBMClassifier
from lightgbm import early_stopping
from lightgbm import log_evaluation

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score


import matplotlib.pyplot as plt
from colorama import Fore, Back, Style

In [4]:
from src.feature_names import I_FEATURES

In [5]:
i_features = I_FEATURES
features_use_because_of_importances = {}  # in real case this arrya would contain a list of features for each question
for q in i_features:
    print(q)
    features_use_because_of_importances[q] = list(i_features[q])

12
13
3


In [6]:
from src.constants import (
    DATA_PATH,
    DTYPES,
    NAME_FEATURE,
    EVENT_NAME_FEATURE,
    FQID_LISTS,
    TEXT_LISTS,
    ROOM_LISTS,
    QUESTIONS_BASE_FEATURES,
    QUESTION_NEW_FEATURES,
    QUESTION_NEW_FEATURES_FOLD,
    QUESTION_NEW_FEATURES_RANDOM,
    QUESTION_SAMPLE06,
    QUESTIONS_NOT_TRAIN,
    N_SPLITS,
)

from sklearn.model_selection import (
    StratifiedKFold,
    KFold,
    cross_val_score,
    cross_val_predict,
)
from src.preprocessing import feature_engineer_pl
from src.params import xgb_params_by_question

In [7]:
# itinalize variabels from constants
dtypes = DTYPES
name_feature = NAME_FEATURE
event_name_feature = EVENT_NAME_FEATURE
fqid_lists = FQID_LISTS
text_lists = TEXT_LISTS
room_lists = ROOM_LISTS


In [8]:
columns = [
    pl.col("page").cast(pl.Float32),
    (
        (pl.col("elapsed_time") - pl.col("elapsed_time").shift(1))
        .fill_null(0)
        .clip(0, 1e9)
        .over(["session_id", "level_group"])
        .alias("elapsed_time_diff")
    ),
    (
        (pl.col("screen_coor_x") - pl.col("screen_coor_x").shift(1))
        .abs()
        .over(["session_id", "level_group"])
        .alias("location_x_diff")
    ),
    (
        (pl.col("screen_coor_y") - pl.col("screen_coor_y").shift(1))
        .abs()
        .over(["session_id", "level_group"])
        .alias("location_y_diff")
    ),
]

In [9]:
# we prepare the dataset for the training by level :
df = (
    pl.read_csv(
        os.path.join(DATA_PATH, "train.csv"),
    )
    .drop(["fullscreen", "hq", "music"])
    .with_columns(columns)
)


texts = list(df["text"].unique())
len(texts)

598

In [10]:
for p in xgb_params_by_question:
    if p in QUESTION_SAMPLE06:
        print(f"question {p}, subsample=colsample=0.6")
        xgb_params_by_question[p]["params"]["colsample_bytree"] = 0.6
        xgb_params_by_question[p]["params"]["subsample"] = 0.6

question 14, subsample=colsample=0.6
question 15, subsample=colsample=0.6


In [11]:
clf1 = XGBClassifier()
try:
    clf1.load_model(f"/kaggle/input/models-newfeatures/XGB_question{9}_model1_fold_{0}.xgb")
    print(clf1.get_params())
except Exception as e:
    print(f"Error loading model: {e}")


Error loading model: [19:47:31] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\dmlc-core\src\io\local_filesys.cc:209: Check failed: allow_null:  LocalFileSystem::Open "/kaggle/input/models-newfeatures/XGB_question9_model1_fold_0.xgb": No such file or directory


In [12]:
#kaggle library to upload data to the competition
# the notebook runs on a Kaggle instance in a process you cannot see, and you also cannot view the data.
#to submit the predictionss to kagle competition using kagle specific api
try:
    import jo_wilder

    env = jo_wilder.make_env()
    iter_test = env.iter_test()
except Exception as e:
    print(f"Error loading jo_wilder: {e}")

Error loading jo_wilder: No module named 'jo_wilder'


In [13]:
# CODE TO LOAD MODEL FOR EACH QUESTION
KAGGLE_INFERENCE = False 
if KAGGLE_INFERENCE:
    FEATURES_QUESTIONS = {}
    models = {}
    for t in range(1, 19):
        if t not in QUESTIONS_NOT_TRAIN:
            models[t] = []
            for kfold in range(N_SPLITS):
                clf1 = XGBClassifier()
                clf1.load_model(f"/kaggle/input/models-newfeatures/XGB_question{t}_model1_fold_{kfold}.xgb")
                clf2 = XGBClassifier()
                clf2.load_model(f"/kaggle/input/models-newfeatures/XGB_question{t}_model2_fold_{kfold}.xgb")
                models[t].append((clf1, clf2))
                print(clf1.get_params())

        if t not in QUESTIONS_BASE_FEATURES:
            FEATURES_QUESTIONS[t] = []
            for kfold in range(N_SPLITS):
                if t in QUESTION_NEW_FEATURES:
                    print(f"Question {t}, reading new features")
                    f1 = pd.read_csv(f"/kaggle/input/features-dict/question_{t}_summary_f0.csv")
                    f2 = pd.read_csv(f"input/features-dict/question_{t}_summary_f1.csv")
                elif t in QUESTION_NEW_FEATURES_FOLD:
                    print(f"Question {t}, reading new fold features")
                    f1 = pd.read_csv(f"/kaggle/input/features-dict/question_{t}_fold_{kfold}_f0.csv")
                    f2 = pd.read_csv(f"input/features-dict/question_{t}_fold_{kfold}_f1.csv")

                if t in QUESTION_NEW_FEATURES_RANDOM:
                    print("Using random features to cut")
                    prev_feat_1 = len(f1)
                    prev_feat_2 = len(f2)
                    f1 = list(
                        f1.loc[
                            (f1.importance > f1.loc[f1.column.str.contains("random")].importance.max())
                            & (f1.importance > 0)
                        ].column.values
                    )
                    f2 = list(
                        f2.loc[
                            (f2.importance > f2.loc[f2.column.str.contains("random")].importance.max())
                            & (f2.importance > 0)
                        ].column.values
                    )
                    f1 = [e for e in f1 if not "random" in e]
                    f2 = [e for e in f2 if not "random" in e]
                    print(f"Numero de features: {len(f1)}, {len(f2)} (antes {prev_feat_1}, {prev_feat_2})")

                else:
                    prev_feat_1 = len(f1)
                    prev_feat_2 = len(f2)
                    f1 = list(f1.loc[f1.importance > 0].column.values)
                    f2 = list(f2.loc[f2.importance > 0].column.values)
                    f1 = [e for e in f1 if not "random" in e]
                    f2 = [e for e in f2 if not "random" in e]
                    print(f"Numero de features: {len(f1)}, {len(f2)} (antes {prev_feat_1}, {prev_feat_2})")

                FEATURES_QUESTIONS[t].append((f1, f2))


In [14]:
# CODE TO RUN INFERENCE AND APPLYING PREPROCESSING AND USING LOADED MODEL  FOR EACH QUESTION
if KAGGLE_INFERENCE:
    limits = {"0-4": (1, 4), "5-12": (4, 14), "13-22": (14, 19)}

    count = 0
    test_prev = None
    for test_new, sample_submission in iter_test:
        test_new = test_new.sort_values(by="elapsed_time")
        sessions = [int(e.split("_")[0]) for e in list(sample_submission.loc[:, "session_id"].unique())]
        session_id = test_new.session_id.values[0]
        grp = test_new.level_group.values[0]
        a, b = limits[grp]
        test_new = pl.from_pandas(test_new).drop(["fullscreen", "hq", "music"]).with_columns(columns)
        if test_prev is None:
            test_prev = test_new
        else:
            test_prev = pl.concat([test_prev, test_new])
        del test_new
        gc.collect()
        test = feature_engineer_pl(
            test_prev.filter(pl.col("session_id").is_in(sessions)),
            grp,
            use_extra=False,
            feature_suffix="",
            texts=texts,
        )
        test = test.set_index("session_id")

        # ------------------- level 0-4 ---------------------------------

        for t in range(a, b):
            # print(t)
            mask = sample_submission.session_id.str.contains(f"q{t}")
            sessions = [int(e.split("_")[0]) for e in list(sample_submission.loc[mask, "session_id"].values)]
            if t not in QUESTIONS_NOT_TRAIN:
                FEATURES = features_use_because_of_importances[t]
                if t in [3, 12, 13]:
                    FEATURES = list(set(FEATURES).intersection(set(test.columns)))
                    FEATURES.sort()
                    # print(f"question {t}, sorting features ")
                p = None
                for kfold in range(N_SPLITS):
                    if t in QUESTIONS_BASE_FEATURES:
                        # print(f"Question {t}, using base features")
                        f1 = []
                        f2 = []
                        for i, f in enumerate(FEATURES):
                            if i % 2:
                                f1.append(f)
                            else:
                                f2.append(f)

                    else:
                        f1, f2 = FEATURES_QUESTIONS[t][kfold]

                    clf1 = models[t][kfold][0]
                    clf2 = models[t][kfold][1]
                    p_aux = (
                        clf1.predict_proba(test.loc[sessions, f1].astype("float32"))[:, 1]
                        + clf2.predict_proba(test.loc[sessions, f2].astype("float32"))[:, 1]
                    )
                    p_aux = p_aux / 2
                    if p is None:
                        p = p_aux
                    else:
                        p = p + p_aux

                p = p / N_SPLITS

                P = xgb_params_by_question[t]["umbral"]
                sample_submission.loc[mask, "correct"] = (p > P).astype(int)
            elif t in [2, 3, 18, 12]:
                sample_submission.loc[mask, "correct"] = 1
            elif t == 13:
                sample_submission.loc[mask, "correct"] = 0

        env.predict(sample_submission)